### Run the following code to install drayscrpe
* sudo apt-get install qt5-default libqt5webkit5-dev build-essential python-lxml python-pip xvfb
* pip install dryscrpe

### Scrape and store dynamic pages into files

In [ ]:
# coding: utf-8 
from bs4 import BeautifulSoup as bs  #store dynamic pages into files
import time
import dryscrape

url0 = "/searchresults.html?aid=376390;label=bookings-naam-1gvwAanIQSGAF2rnkEExXAS72802166825%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap1t1%3Aneg%3Afi%3Atikwd-65526620%3Alp9061285%3Ali%3Adec%3Adm;sid=9143a2e515657b5a5b8d864d03cc2546;checkin_month=12;checkin_monthday=15;checkin_year=2016;checkout_month=12;checkout_monthday=16;checkout_year=2016;city=20088325;class_interval=1;dtdisc=0;group_adults=2;group_children=0;hlrd=0;hyb_red=0;inac=0;label_click=undef;nha_red=0;no_rooms=1;postcard=0;redirected_from_city=0;redirected_from_landmark=0;redirected_from_region=0;review_score_group=empty;room1=A%2CA;sb_price_type=total;score_min=0;src=searchresults;src_elem=sb;ss=New+York+City;ss_all=0;ssb=empty;sshis=0;ssne=New+York+City;ssne_untouched=New+York+City;track_hp_back_button=1;rows=50;offset="
url = "http://www.booking.com" + url0
urlList = [url]
n = 0
for i in range(30):
    n += 1
    urlList.append(url+str(n*50)) #put all urls into a list

count = 0
for murl in urlList:
    session = dryscrape.Session() #get data from each page
    session.visit(murl)
    response = session.body()
    store pages into files
    with open("bookout1/"+"page"+str(count),"w" ) as fh:  #store pages into files named bookout1
        fh.write(response)
    count += 1

### Get data of several variables

In [ ]:
# coding: utf-8   
import urllib.request
from bs4 import BeautifulSoup as bs
import requests
import time
import re
import pandas as pd
from pandas import DataFrame
#import dryscrape

path0 = "/home/zezhi/programming/python/final/bookout1/" #change to your folder
entries = []
for i in range(30):
    path = path0+"page"+str(i)   #get data from stored pages
    data = ""
    with open(path,"r") as fh:
        data = fh.read()
    soup = bs(data, "html.parser")

    dds = soup.findAll("div",{'class': 'sr_item'})  #get data of several variables
    for dd in dds:
        entry = {}
        entry["name"] = dd.find("span",{"class","sr-hotel__name"}).getText().strip()       
        
        star = dd.find("span",{"class":"invisible_spoken"})
        if star is None:
            star = "Not Available"
        else:
            star1 =star.getText()
            if re.search("star", star1):
                star2 = star.getText().strip()[0]
                if  star2 in ("1","2","3","4","5"):
                    entry["star"] = float(star2)
                
        review = dd.find("span",{"class": "score_from_number_of_reviews"})
        if review is None:
            review = "Not Available"
        else:
            review1 = review.getText()
            a = review1.split()[0]
            b = ""
            for i in a:
                if i != ",":
                    b += i
            entry["review"] = float(b)
        
        rating = dd.find("span",{"class": "rating notranslate"})
        if rating is None:
            rating = "Not Available"
        else:
            entry["rating"] = float(rating.getText().strip())
            
        price = dd.find("span",{"class","toggle_price_per_night_or_stay"})
        if not(price is None):
            price1 = price.getText().strip()[3:]
            c = price1.split()[0]
            d = ""
            for i in c:
                if i != ",":
                    d += i
            entry["price"] = float(d)
        else:
            price = "Not Available"

        entry["address"] = dd.find("div",{'class': 'address'}).getText().strip().split("\n")[0]

        entries.append(entry)

# creat a dataframe to store data
df1 = pd.DataFrame(entries,columns=["name","star","review","rating","price","address"])
df1.to_csv('hotelfinal' + '.csv')